In [1]:
import os
import numpy as np

from keras.models import load_model

#Load Pre-Trained FaceNet

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
model_dir = '/content/drive/MyDrive/L12/'
os.chdir(model_dir)

# load the model
model = load_model('Cher Eng Lim - facenet_keras.h5')

##Activity 1: Information of the pre-trained model

- Number of layers:

In [8]:
print("Number of layer = ", len(model.layers))

Number of layer =  426


- Input:

In [11]:
print("Input = ", model.input_shape)

Input =  (None, 160, 160, 3)


The information tells you that we need to feed images of size $160\times 160$ pixels. Images are colored ones (RGB)

- Ouput

In [12]:
print("Output = ", model.output_shape)

Output =  (None, 128)


After feeding an image to the model, the output of the model is an array of length $128$. 

#Load a face and feed it the the pre-trained model.

Change directory to the one containing some faces.

In [13]:
face_dir = '/content/drive/MyDrive/Cher Eng Lim - faces.zip (Unzipped Files)/faces'
os.chdir(face_dir)
%ls

face01.PNG  face03_02.PNG  face04.PNG  face06.PNG  face08.PNG     face09.PNG
face02.PNG  face03.PNG     face05.PNG  face07.PNG  face09_02.PNG  face10.PNG


We will use `Image` module of `Pillow`, Python Imaging Library, to handle images.

In [14]:
import PIL.Image as Image

Load `face10.PNG` and print out the size of the image.

In [15]:
# load the image and convert to RGB
face10 = Image.open('face10.PNG').convert("RGB")

# convert it to numpy array
face10_np = np.asarray(face10)

# print the size
print("Size of the face:", face10_np.shape)

Size of the face: (409, 291, 3)


The size of the face is not $160\times 160$. Therefore, we need to resize it (using `Image` module) before converting it numpy array.

In [19]:
# load the image and convert to RGB
face10 = Image.open('face10.PNG').convert("RGB")

#resize
required_size = (160,160)
face10_resized = face10.resize(required_size)

# convert it to numpy array
face10_np = np.asarray(face10_resized)

# print the size
print("Size of the face:", face10_np.shape)

Size of the face: (160, 160, 3)


Before feeding it to the model we need to do normalize the face.

In [17]:
# just to make sure the pixels are of float32 type
face10_np = face10_np.astype('float32')

# standardize pixel values across channels
mean, std = face10_np.mean(), face10_np.std()
print("Mean = ", mean)
print("STD = ", std)

face10_np = (face10_np - mean) / std

Mean =  125.16738
STD =  50.166664


In [18]:
# transform the face into (1, 160, 160, 3)
face10_np_expanded = np.expand_dims(face10_np, axis = 0)
print("Size after expansion: ", face10_np_expanded.shape)

# getting the embeddings of the face
face10_embedding = model.predict(face10_np_expanded)

print("\nType of the embedding: ", type(face10_embedding))
print("Size of the embedding: ", face10_embedding.shape)

Size after expansion:  (1, 160, 160, 3)

Type of the embedding:  <class 'numpy.ndarray'>
Size of the embedding:  (1, 128)


We have sucessfully covert a face into an array of $128$ numbers.

##Activity 2: Write a function to convert a numpy array of size $160\times 160\times 3$ into an array of length $128$

In [20]:
def convert_to_embedding(face, model):

  face = face.astype('float32')
  
  mean, std = face.mean(), face.std()
  face = (face - mean) / std

  face_expanded = np.expand_dims(face, axis=0)

  face_embedding = model.predict(face_expanded)

  return face_embedding

Call the function to calculate the embedding of `face10.PNG` and calculate its L2-norm

Note: 

For a vector of $n$ elements, $\boldsymbol{x}=[x_1,\ x_2, \ \cdots,\ x_n]^T$ then its L2-norm is

$|\boldsymbol{x}|_2=\sqrt{x_1^2+x_2^2+\cdots+x_n^2}$

We can use [`np.np.linalg.norm()`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html) to calculate this norm.

In [21]:
face10 = Image.open('face10.PNG').convert("RGB")
face10_resized = face10.resize(required_size)
face10_np = np.asarray(face10_resized)

face10_embedding = convert_to_embedding(face10_np, model)

In [22]:
print("L2-norm of the embedding = ", np.linalg.norm(face10_embedding))

L2-norm of the embedding =  10.799945


#Compare embeddings of different and similar faces.

##Activity 3
- Calculate the embeedings of `face02.PNG`

- Calculate the embeedings of `face03.PNG` and `face03_02.PNG`

- Calculate the L2-norm of

  - The difference of embeedings of `face02.PNG` and `face03.PNG`.
  - The difference of embeedings of `face03.PNG` and `face03_02.PNG`.

In [23]:
face02 = Image.open('face02.PNG').convert("RGB")
face02_resized = face02.resize(required_size)
face02_np = np.asarray(face02_resized)

face02_embedding = convert_to_embedding(face02_np, model)

#face02_embedding = ........................

In [24]:
print("L2-norm of the embedding = ", np.linalg.norm(face02_embedding))

L2-norm of the embedding =  9.374028


In [25]:
face03 = Image.open('face03.PNG').convert("RGB")
face03_resized = face03.resize(required_size)
face03_np = np.asarray(face03_resized)

face03_embedding = convert_to_embedding(face03_np, model)

#face03_embedding = ........................

face03_02 = Image.open('face03_02.PNG').convert("RGB")
face03_02_resized = face03_02.resize(required_size)
face03_02_np = np.asarray(face03_02_resized)

face03_02_embedding = convert_to_embedding(face03_02_np, model)

#face03_02_embedding = ........................

In [26]:
print("L2-norm distance of face02 and face03 = ", np.linalg.norm(face02_embedding - face03_embedding))

L2-norm distance of face02 and face03 =  13.809298


In [27]:
print("L2-norm distance of face03 and face03_02 = ", np.linalg.norm(face03_embedding - face03_02_embedding))

L2-norm distance of face03 and face03_02 =  6.2999187


#Optional

Here is a simple way to get names of files in a directory together with their indices.

In [28]:
from os import walk
face_dir = '/content/drive/MyDrive/Cher Eng Lim - faces.zip (Unzipped Files)/faces'
(_, _, filenames) = next(os.walk(face_dir))

for i, name in zip(range(len(filenames)), filenames):
  print(i)
  print(name)

0
face03_02.PNG
1
face02.PNG
2
face01.PNG
3
face03.PNG
4
face04.PNG
5
face05.PNG
6
face06.PNG
7
face07.PNG
8
face08.PNG
9
face09.PNG
10
face09_02.PNG
11
face10.PNG


Let's wrtite a simple code to read each file, calculate its embeddings and store the result into a matrix.

In [29]:
embedding_matrix = np.empty((len(filenames), 128))

for i, name in zip(range(len(filenames)), filenames):
  
  face = Image.open(name).convert("RGB")

  face_resized = face.resize(required_size)

  face_np = np.asarray(face_resized)

  embedding_matrix[i, :] = convert_to_embedding(face_np, model)

Calcualte the pairwise distance of the resultant matrix.

Reference: https://jbencook.com/pairwise-distance-in-numpy/

In [30]:
from scipy.spatial import distance_matrix

result = distance_matrix(embedding_matrix, embedding_matrix)

# https://numpy.org/doc/stable/reference/generated/numpy.printoptions.html
# suppress = True --> use scientfic notation for small numbers
with np.printoptions(precision = 2, suppress = True): 
    print(result)

[[ 0.   13.8  13.44  6.3  15.91 14.35 14.87 14.95 15.81 13.73 13.91 15.25]
 [13.8   0.   13.25 13.81 11.45 14.8  15.3  16.42 12.92 13.1  11.96 14.55]
 [13.44 13.25  0.   14.61 15.5  14.21 14.83 15.15 15.46 14.27 14.07 14.87]
 [ 6.3  13.81 14.61  0.   15.9  15.49 15.47 15.67 15.97 13.82 14.07 15.01]
 [15.91 11.45 15.5  15.9   0.   15.51 15.92 15.92 14.41 15.27 14.38 16.28]
 [14.35 14.8  14.21 15.49 15.51  0.   14.75 13.76 15.46 16.13 14.71 14.81]
 [14.87 15.3  14.83 15.47 15.92 14.75  0.   14.48 17.23 13.79 14.97 17.01]
 [14.95 16.42 15.15 15.67 15.92 13.76 14.48  0.   15.56 16.03 15.31 14.55]
 [15.81 12.92 15.46 15.97 14.41 15.46 17.23 15.56  0.   13.95 13.74 13.97]
 [13.73 13.1  14.27 13.82 15.27 16.13 13.79 16.03 13.95  0.    7.91 15.33]
 [13.91 11.96 14.07 14.07 14.38 14.71 14.97 15.31 13.74  7.91  0.   14.98]
 [15.25 14.55 14.87 15.01 16.28 14.81 17.01 14.55 13.97 15.33 14.98  0.  ]]
